In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

import time
import torch
import torch.nn as nn
import os
from PIL import Image
import numpy as np
import glob
from random import randint, shuffle
from io import BytesIO
from torch.autograd import Variable
import torchvision.transforms as transforms
import cv2
def resize_image(im,bandwidth=256):
    
    img = im.resize((bandwidth, bandwidth),  Image.ANTIALIAS)
    img=np.array(img)/255*2-1
    img=img[:,:,:3]
    img = np.moveaxis(img, 2, 0)
    return img

class UBlock(nn.Module):
    def __init__(self, s, nf_in, max_nf, use_batchnorm=True, nf_out=None, nf_next=None):
        super(UBlock, self).__init__()
        #print("Initializing ublock- size of s =", s)
        assert s>=2 and s%2==0
        nf_next = nf_next if nf_next else min(nf_in*2, max_nf)
        nf_out = nf_out if nf_out else nf_in            
        self.conv = nn.Conv2d(nf_in, nf_next, 4, 2, 1, bias=not (use_batchnorm and s>2) )
        if s>2:
            next_block = [nn.BatchNorm2d(nf_next)] if use_batchnorm else []
            next_block += [nn.LeakyReLU(0.2, inplace=True), UBlock(s//2, nf_next, max_nf)]
            self.next_block = nn.Sequential(*next_block)
        else:
            self.next_block = None
        convt = [nn.ReLU(), 
                 nn.ConvTranspose2d(nf_next*2 if self.next_block else nf_next, nf_out,
                                        kernel_size=4, stride=2,padding=1, bias=not use_batchnorm)]    
        if use_batchnorm:
            convt += [nn.BatchNorm2d(nf_out)]        
        if s <= 8:
            convt += [nn.Dropout(0.5, inplace=True)]
        self.convt = nn.Sequential(*convt)  

    def forward(self, x):
        x = self.conv(x)
        if self.next_block:
            
            x2 = self.next_block(x)
            x = torch.cat((x,x2),1)
        return self.convt(x)        


def UNET_G(isize, nc_in=3, nc_out=3, ngf=64):
    return nn.Sequential(
                  UBlock(isize, nc_in, 8*ngf, False, nf_out=nc_out, nf_next=ngf),
                  nn.Tanh() )

def netG_gen(A):
    return netG(A) 
imageSize=256
netG = UNET_G(isize=256, nc_in=3, nc_out=3, ngf=64)
netG.cuda()
device = torch.device('cuda')

if os.path.exists("/content/drive/My Drive/sceneGen/Generator.pt"):
  netG=torch.load("/content/drive/My Drive/sceneGen/Generator.pt", map_location="cuda:0")
  netG.to(device=device,dtype=torch.float)


print("model ready")

model ready


In [0]:
import glob
anuepath="/content/drive/My Drive/anue_eval"
Fake=glob.glob(anuepath+'/gtFine/*_labelColors.png')
input_paths=sorted(Fake)


In [0]:
i=0
for path_to_input_img in input_paths:
  image = Image.open(path_to_input_img,'r')
  image=resize_image(image)
  image=np.expand_dims(image, axis=0)
  
  #making input into pytorch variable
  
  image=Variable(torch.from_numpy(image).cuda())
  #getting output from network
  output= netG_gen(image.float()).cpu().detach().numpy()
  #manipulating output image
  int_X = ( (output+1)/2*255).clip(0,255).astype('uint8')
  int_X = np.moveaxis(int_X.reshape(-1,3,imageSize,imageSize), 1, 3)
  int_X = int_X.reshape(-1, imageSize, imageSize,3).reshape(imageSize,-1, 3)
  path_to_output_img = anuepath+"/generated/"+path_to_input_img[41:path_to_input_img.index("_gtFine")]+"_generated.jpg"
  response=Image.fromarray(int_X).resize((1280,964),Image.ANTIALIAS)
  response.save(path_to_output_img)
  if i%100==0:
    print(i)
  i+=1

0
100
200
300
400
500
600
700
800
900
